<h1>Snow Clearance Fines, 2019-2023</h1>
31 January 2024

This analysis looks at fines levied for uncleared sidewalks, based on FOIA data requested from the Department of Administrative Hearings (H064920-011124.xlsx). This dataset contained 3058 records dating from 1/1/2001 to 9/12/2023; filtered for those between 7/1/2019 and 6/30/2023 has 2560 records. Four of these could not be geocoded due to "unknown" address.<br>

<div style="color:red">this needs to be updated to reflect re-built data pipeline</div>
<br>
My analysis steps:
<ol>
<li><a href="#read">Read Data</a>
    <li><a href="#summarize">Summarize</a>- by issuing department, by year, by community area
        <li><a href="#community">Community Summary</a>
</ol>

### Preliminary Findings
<ul>
    <li>Englewood, Garfield Ridge, and West Englewood are the three communities with the highest number of dockets per capita
    <li>17 of 25 police-related dockets related to snow clearance were in Englewood, West Englewood, Garfield Ridge, and Belmont Cragin
    </ul>

<a name="read"></a>
# 1. Read Data

In [1]:
import pandas as pd
# import requests
import numpy as np
import altair as alt
#import datetime as dt #would only need this if I manipulated dates post-API data retrieval

In [2]:
df = pd.read_csv("../../data/05-finalized/dockets-to-details-gis.csv")
df.head()

,respondent,docket,violation_num,violation_date,hearing_date,violation_address,violation_desc,case_disposition,fine_amt,dept,cleaned_address,latlong,community,ward_1523,lat,long
0,1010 W MADISON LLC,23DT000582,T000215159,2023-01-31,7/10/2023,1010 W MADISON,10-8-180 Snow and ice removal.,Non-Suit,0.0,TRANPORT,1010 W MADISON,"41.88196975,-87.6524227995416",NEAR WEST SIDE,25,41.88196975,-87.6524227995416
1,"1100 E 47TH STREET LLC, C/O PETER CASSEL",22DT001042,T000207342,2022-01-04,5/23/2022,1100 E 47TH ST,10-8-180 Snow and ice removal.,Non-Suit,0.0,TRANPORT,1100 E 47TH ST,"41.809698,-87.599101",KENWOOD,4,41.809698,-87.599101
2,"1101 BERWYN LLC, C/O JOHN BARRETT",23DT000559,T000220112,2023-02-02,3/20/2023,1101 W BERWYN,10-8-180 Snow and ice removal.,Non-Suit,0.0,TRANPORT,1101 W BERWYN,"41.978248,-87.65863472496416",EDGEWATER,48,41.978248,-87.65863472496416
3,1157 W ERIE,22DT001387,T000207431,2022-02-08,4/11/2022,100 UNKNOWN,10-8-180 Snow and ice removal.,Not Liable,0.0,TRANPORT,NaN,NaN,unknown,unknown,unknown,unknown
4,1200B INC C/O THOMAS GEORGE,21DS20339M,QTUF62L,2021-02-05,7/14/2021,1200 W BELMONT AVE,10-8-180 Snow and ice removal.,Not Liable,0.0,STRTSAN,1200 W BELMONT AVE,"41.9401529,-87.659326",LAKE VIEW,44,41.9401529,-87.659326


In [3]:
len(df)

3662

<a name="summarize"></a>
# 2. Summarize Citywide

### by season

In [4]:
# by season
df.groupby('season').agg(
    sum_fine_amt=('Imposed Fine Detailed', 'sum'),
    max_fine=('Imposed Fine Detailed', 'max'),
    n_dockets=('Docket Number','nunique'),
    n_records=('Docket Number','count')
).reset_index()

KeyError: 'season'

### by issuing department

In [7]:
# by issuing department
df.groupby('Issuing Department Code').agg(
    sum_fine_amt=('Imposed Fine Detailed', 'sum'),
    n_dockets=('Docket Number','nunique'),
    n_records=('Docket Number','count')
).reset_index()

KeyError: 'Issuing Department Code'

### by Address

In [6]:
df_by_address = df.groupby(['Violation Address','community']).agg(
    sum_fine_amt=('Imposed Fine Detailed', 'sum'),
    n_dockets=('Docket Number','nunique'),
    n_records=('Docket Number','count')
).reset_index()
df_by_address[df_by_address['n_dockets']>=3].sort_values("n_dockets",ascending=False).head()

,Violation Address,community,sum_fine_amt,n_dockets,n_records
714,3110 W 61ST ST,CHICAGO LAWN,0.0,6,6
1717,932 W 59TH ST,ENGLEWOOD,8500.0,4,10
1422,627 W RANDOLPH ST,NEAR WEST SIDE,1300.0,4,5
1054,4710 S WESTERN AVE,BRIGHTON PARK,1400.0,4,6
1034,4601 S ROCKWELL ST,BRIGHTON PARK,1000.0,3,4


### distribution of fines by year

In [7]:
bins = [0, 1, 150, 500, 501, 5000]
labels = ['0', '1-150', '151-499', '500', '501-5000']

df['Binned Fine'] = pd.cut(df['Imposed Fine Detailed'], bins=bins, labels=labels, right=False, include_lowest=True)

pivot_table = df.pivot_table(
    values='Imposed Fine Detailed',  # We sum over the original fine amounts
    index='Binned Fine',  # Use the binned column as the new index
    columns='season',  # Using 'season' for the columns
    aggfunc='count',  # Summing up the fine amounts
    fill_value=0  # Replace NaN with 0
)

pivot_table

season,2019-2020,2020-2021,2021-2022,2022-2023
Binned Fine,,,,
0,159,186,168,27
1-150,50,140,95,14
151-499,200,303,382,41
500,108,355,244,38
501-5000,9,10,22,4


In [8]:
pivot_table_long = pivot_table.reset_index().melt(id_vars='Binned Fine', var_name='Season', value_name='Count')

# Create the histogram (bar chart) with Altair
chart = alt.Chart(pivot_table_long).mark_bar().encode(
    x=alt.X('Binned Fine:N', title='Fine Amount Bins', sort=labels),  # Ensure custom order
    y=alt.Y('sum(Count):Q', title='Count'),
    color='Season:N',
    column='Season:N'  # Separate charts for each season
).properties(
    width=220,
    height=200
).resolve_scale(
    y='independent'  # This allows each chart to have a separate Y-axis scale
).properties(
    width=220,
    height=200
)

chart.display()

/home/robcat26/.local/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

<a name="community"></a>
# 3. Summarize by Community

### by community

In [9]:
df_community = df.groupby('community').agg(
    sum_fine_amt=('Imposed Fine Detailed', 'sum'),
    n_dockets=('Docket Number','nunique'),
    n_records=('Docket Number','count')
).reset_index()
df_community.head()

,community,sum_fine_amt,n_dockets,n_records
0,ALBANY PARK,3360.0,15,18
1,ARCHER HEIGHTS,3580.0,12,15
2,ARMOUR SQUARE,3990.0,26,33
3,ASHBURN,200.0,2,2
4,AUBURN GRESHAM,9250.0,19,29


In [10]:
df_community.dtypes

community        object
sum_fine_amt    float64
n_dockets         int64
n_records         int64
dtype: object

### read community population

In [11]:
# retrieved on 1/11/24, but 2020 Census Population figures should be static
df_population = pd.read_csv("../data/01-raw/population_cmap_2022.csv")

In [12]:
# simplify dataframe to get only essentials
df_population = df_population[['GEOID','GEOG','2020_POP']]
df_population = df_population.rename(columns={'GEOG':'COMMUNITY_NAME'})
df_population['COMMUNITY_CAPS']=df_population['COMMUNITY_NAME'].str.upper()
df_population.head()

,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS
0,14,Albany Park,48396,ALBANY PARK
1,57,Archer Heights,14196,ARCHER HEIGHTS
2,34,Armour Square,13890,ARMOUR SQUARE
3,70,Ashburn,41098,ASHBURN
4,71,Auburn Gresham,44878,AUBURN GRESHAM


In [13]:
df_population.dtypes

GEOID              int64
COMMUNITY_NAME    object
2020_POP           int64
COMMUNITY_CAPS    object
dtype: object

### merge in community population data

In [14]:
df_community_summary = pd.merge(df_community,df_population,left_on='community',right_on='COMMUNITY_CAPS')
df_community_summary.head()

,community,sum_fine_amt,n_dockets,n_records,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS
0,ALBANY PARK,3360.0,15,18,14,Albany Park,48396,ALBANY PARK
1,ARCHER HEIGHTS,3580.0,12,15,57,Archer Heights,14196,ARCHER HEIGHTS
2,ARMOUR SQUARE,3990.0,26,33,34,Armour Square,13890,ARMOUR SQUARE
3,ASHBURN,200.0,2,2,70,Ashburn,41098,ASHBURN
4,AUBURN GRESHAM,9250.0,19,29,71,Auburn Gresham,44878,AUBURN GRESHAM


In [15]:
# per 10,000 capita, per year over 4 years
df_community_summary['dockets per 10k'] = \
(10000/4)*df_community_summary['n_dockets']/df_community_summary['2020_POP']

In [16]:
df_community_summary

,community,sum_fine_amt,n_dockets,n_records,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS,dockets per 10k
0,ALBANY PARK,3360.0,15,18,14,Albany Park,48396,ALBANY PARK,0.774857
1,ARCHER HEIGHTS,3580.0,12,15,57,Archer Heights,14196,ARCHER HEIGHTS,2.113271
2,ARMOUR SQUARE,3990.0,26,33,34,Armour Square,13890,ARMOUR SQUARE,4.679626
3,ASHBURN,200.0,2,2,70,Ashburn,41098,ASHBURN,0.121660
4,AUBURN GRESHAM,9250.0,19,29,71,Auburn Gresham,44878,AUBURN GRESHAM,1.058425
...,...,...,...,...,...,...,...,...,...
65,WEST LAWN,5750.0,17,23,65,West Lawn,33662,WEST LAWN,1.262551
66,WEST PULLMAN,1450.0,5,9,53,West Pullman,26104,WEST PULLMAN,0.478854
67,WEST RIDGE,4800.0,24,28,2,West Ridge,77122,WEST RIDGE,0.777988
68,WEST TOWN,27565.0,97,127,24,West Town,87781,WEST TOWN,2.762557


In [17]:
df_community_summary.sort_values('dockets per 10k',ascending=False).head(10)

,community,sum_fine_amt,n_dockets,n_records,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS,dockets per 10k
20,ENGLEWOOD,44960.0,69,107,68,Englewood,24369,ENGLEWOOD,7.078666
24,GARFIELD RIDGE,21980.0,91,122,56,Garfield Ridge,35439,GARFIELD RIDGE,6.419481
63,WEST ENGLEWOOD,27130.0,74,102,67,West Englewood,29647,WEST ENGLEWOOD,6.240092
25,GRAND BOULEVARD,16770.0,52,71,38,Grand Boulevard,24589,GRAND BOULEVARD,5.286917
50,OAKLAND,4170.0,13,19,36,Oakland,6799,OAKLAND,4.780115
17,EAST GARFIELD PARK,11460.0,38,54,27,East Garfield Park,19992,EAST GARFIELD PARK,4.751901
2,ARMOUR SQUARE,3990.0,26,33,34,Armour Square,13890,ARMOUR SQUARE,4.679626
35,LINCOLN PARK,40739.0,128,168,7,Lincoln Park,70492,LINCOLN PARK,4.539522
10,BRIGHTON PARK,22340.0,81,105,58,Brighton Park,45053,BRIGHTON PARK,4.494706
33,KENWOOD,9430.0,33,46,39,Kenwood,19116,KENWOOD,4.315756


In [18]:
df_community_summary.to_csv("../results/ssw-part1-admin-hearings-by-community.csv")